In [1]:
suppressMessages(library(MuSiC))
suppressMessages(library(Seurat))
suppressMessages(library(fastSave))
suppressMessages(library(pbmcapply))
suppressMessages(library(SingleCellExperiment))

## 数据导入及bulk数据整理

In [2]:
ref_seurat=readRDS.lbzip2('~/ReCIDE/benchmark测试/mult_reference_PBMC/ref_data/ref_all.rdsFS',n.cores = 200)
EXP<- readRDS("~/ReCIDE/benchmark测试/mult_reference_PBMC/EXP_and_KEY/EXP_PBMC.rds")

###########################

In [3]:
######构建bulk
bulk.mtx<-as.data.frame(EXP[[1]])
for (i in 2:length(EXP)) {
  bulk.mtx2<-as.data.frame(EXP[[i]])
  bulk.mtx<-cbind(bulk.mtx,bulk.mtx2)
  
}
bulk.mtx<-as.matrix(bulk.mtx)
colnames(bulk.mtx)<-names(EXP)


In [4]:

kk<-as.data.frame(ref_seurat@assays[["RNA"]]@counts)
kksum<-as.data.frame(apply(kk, 2, sum))
all(kksum>0)
kksum<-subset(kksum,kksum[,1]>0,)
kksum<-as.data.frame(kksum)
ref_seurat<-ref_seurat[,row.names(kksum)]
rm(kksum)
gc()


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 28.1 GiB”


[1] TRUE

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,8680424,463.6,12954148,691.9,12954148,691.9
Vcells,5105009307,38948.2,14560359373,111086.8,17031807121,129942.4


In [5]:
head(ref_seurat@meta.data)

,cell_label,batch,nCount_RNA,nFeature_RNA
,<chr>,<chr>,<dbl>,<int>
ref1_AAACCTGAGACCACGA-newcastle65,T4,newcastle65,4131,1255
ref1_AAACCTGAGATGTCGG-newcastle65,NK,newcastle65,4580,1814
ref1_AAACCTGAGGCGATAC-newcastle65,T4,newcastle65,2769,990
ref1_AAACCTGAGTACACCT-newcastle65,NK,newcastle65,5071,2023
ref1_AAACCTGAGTGAATTG-newcastle65,T8,newcastle65,3890,1514
ref1_AAACCTGCATCACGAT-newcastle65,T8,newcastle65,5591,1794


In [6]:
#单细胞参考集是sce的形式
sce <- SingleCellExperiment(as.matrix(ref_seurat@assays$RNA@counts),
                            colData=DataFrame(label=ref_seurat@meta.data),
                            rowData=DataFrame(length=row.names(ref_seurat)))
names(assays(sce))<-'counts'

#在music前要先确定bulk和sc是否只有一列，只有一列的话无法运行（包括sample和cluster数是否为1）
music_output = music_prop(bulk.mtx = bulk.mtx, sc.sce = sce, clusters = 'label.cell_label',
                              samples = 'label.batch',verbose = F)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 28.1 GiB”


In [7]:
saveRDS(music_output,file='/home/lmh/ReCIDE/benchmark测试/mult_reference_PBMC/music/music_tutorial/MuSiC_output.rds')

In [1]:
music_output=readRDS('/home/lmh/ReCIDE/benchmark测试/mult_reference_PBMC/music/music_tutorial/MuSiC_output.rds')
prd_df=as.data.frame(t(music_output$Est.prop.weighted))

In [2]:
# saveRDS(prd_df,file='/home/lmh/ReCIDE/benchmark测试/mult_reference_PBMC/music/music_tutorial/MuSiC_prd_df.rds')

In [3]:
key_df<- readRDS("~/ReCIDE/benchmark测试/mult_reference_PBMC/EXP_and_KEY/key_PBMC.rds")


In [4]:
suppressMessages(library(dplyr))

prd_df=prd_df[sort(row.names(prd_df)),sort(colnames(prd_df))]
key_df=key_df[sort(row.names(key_df)),sort(colnames(key_df))]

# prd_df=prd_df[,c('P1','P2','P3','P4','P5','P6','P7','P8')]
# key_df=key_df[,c('P1','P2','P3','P4','P5','P6','P7','P8')]

rmse_vec=c()
for (i in 1:ncol(key_df)) {
  rmse_vec[i]=ModelMetrics::rmse(prd_df[,i],key_df[,i])
}


In [5]:
mean(rmse_vec)

[1] 0.09746686

In [15]:
saveRDS(prd_df,file='/home/lmh/ReCIDE/benchmark测试/mult_reference_PBMC/music/music_tutorial/MuSiC_prd_df.rds')